In [ ]:
################################################################
# Scirpt_Name: FEWS_PET_Organizing.py                          #
# Purpose(s): 1. Concatenating all .bil FEWS PET datasets      #
#             2. Save as .nc format at local                   #
################################################################
# Written by Lujun Zhang @ U of Oklahoma 06/16/2020            #
# REVISION HISTORY                                             #
# 20200616 -- Initial section one completed by Lujun @ OU      #
################################################################
# Requie packages: Numpy, Pandas, necCDF4                      #
################################################################

In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
from datetime import timedelta
import sys
import os
import time
import datetime
from netCDF4 import Dataset  
from osgeo import gdal, ogr
import matplotlib.pyplot as plt
import matplotlib as mpl 
from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import interp
from multiprocessing import Pool
from ftplib import FTP
import gdal

In [ ]:
def ReadBilFile(bil):
    import gdal
    gdal.GetDriverByName('EHdr').Register()
    img = gdal.Open(bil)
    geotransform = img.GetGeoTransform()
    band = img.GetRasterBand(1)
    data = band.ReadAsArray()

    return data, geotransform

In [ ]:
year_range = range(2019,2020)
for i in year_range:
    ## 
    # Tell if its a leap year(365 days or 366 days)
    if (i%400==0)or(i%4==0 and i%100!=0):
        j_range = 366
    else:
        j_range = 365
    # Creating dates for later str stiching
    date = pd.date_range(start=(str(i)+'/01/01'), end=(str(i)+'/12/31'), freq="d")
    ##
    # Looping within each year "i" daily PRISM precipitation data
    for j in range(j_range):
            date_tab = pd.to_datetime(date[j]).strftime('%Y%m%d')
            if (i==2019 and j>242):
                bil = r'/vsizip/F:\PRISM_Daily_Precipitation\Raw\PRISM_ppt_provisional_4kmD2_{0}_bil.zip\PRISM_ppt_provisional_4kmD2_{0}_bil.bil'.format(date_tab)
            else:
                bil = r'/vsizip/F:\PRISM_Daily_Precipitation\Raw\PRISM_ppt_stable_4kmD2_{0}_bil.zip\PRISM_ppt_stable_4kmD2_{0}_bil.bil'.format(date_tab)       
            data_tab, geotransform = ReadBilFile(bil)
            data_tab = np.flip(data_tab,axis=0)
            data_tab = np.expand_dims(data_tab,0)
            if (j==0):
                ##
                # Reading & Saving GeoReference
                originX = geotransform[0]
                pixelWidth = geotransform[1]
                originY = geotransform[3]
                pixelHeight = geotransform[5]
                Lon = sorted(np.arange(originX,originX+pixelWidth*(np.shape(data_tab)[2]-0.001),pixelWidth)+360)
                Lat = sorted(np.arange(originY+pixelHeight*(np.shape(data_tab)[1]),originY,-pixelHeight))
                ##
                # Creating "Data" to concatenate all PRISM data in one matrix
                Data = data_tab
                print('Start orgnazing year '+str(i)+' PRISM data')
            elif (j==range(j_range)[-1]):
                Data = np.concatenate((Data,data_tab),axis=0) 
                print('Data-set shape '+str(np.shape(Data)))
                ## 
                # Save concatenated PRISM dataset at local disk
                Out_Direc = 'F:/PRISM_Daily_Precipitation/Organized/PRISM_'+str(i)+'_Daily_Precipitation.nc'
                ncOutput = Dataset(Out_Direc, 'w', format='NETCDF4')
                ncOutput.createDimension('Lon', np.size(Lon))
                ncOutput.createDimension('Lat', np.size(Lat))
                ncOutput.createDimension('T', np.shape(Data)[0])
                ##
                # Add lat Variable
                var_out_Y = ncOutput.createVariable('Lat','f',("Lat"))
                ncOutput.variables['Lat'][:] = Lat[:]
                ##
                # Add lon Variable
                var_out_X = ncOutput.createVariable('Lon','f',("Lon"))
                ncOutput.variables['Lon'][:] = Lon[:]
                ##
                # Add time Variable
                var_out_S = ncOutput.createVariable('T','f',("T"))
                ncOutput.variables['T'][:] = np.arange(np.shape(Data)[0])[:]
                ##
                # Add data Variable
                var_out_data = ncOutput.createVariable('Prec', 'f',("T","Lat","Lon"))
                ncOutput.variables['Prec'][:,:,:] = Data[:,:,:]
                ##
                # attr
                ncOutput.history = "Created datatime " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + " by LujunZ at OU"
                ncOutput.source  = "netCDF4 under python 3.6.5"
                ncOutput.close()  # close the new file  
                
                print('year '+str(i)+' PRISM data re-organized!')    
            else:
                Data = np.concatenate((Data,data_tab),axis=0) 